In [1]:
%matplotlib inline
from preamble import *
plt.rcParams['image.cmap'] = "gray"

In [2]:
from importlib import reload
import sys
reload(sys) 

<module 'sys' (built-in)>

In [3]:
import gensim
from gensim.models import doc2vec
from gensim.models.doc2vec import TaggedDocument
from collections import namedtuple
import pandas as pd
import smart_open
import random
import chardet

In [4]:
def read_data(filename): 
    with open(filename, 'r') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:] # header    

    return data
train_data = read_data('8.txt')

In [5]:
df = pd.read_table('8.txt') #16sample interview data

In [6]:
df

,ID,TEXT
0,DSJY,자 저희는 유니스트 소속 대학원 수업 CONTEXTUAL DESGIN 을 수강하고 ...
1,P2,네
2,DSJY,보나는 어떤 집단인지 궁금합니다
3,P2,우리는 습관에 대해서 항상하고 사람들이 움직이는 방법에 대해서 건강한 습관을 가지고...
4,DSJY,굉장히 좋은 취지인것 같습니다
5,P2,그렇죠? 우리팀에서 일할래?
6,DSJY,"돈주면..., 보나의 규모가 어느정도인가요?"
7,P2,"규모라는게 너무 작아서 규모라고 할 정도가 아니고 정말 작아서 그냥 1인기업, 스타..."
8,DSJY,업무환경에 대해서 말씀해 주실수 있나요? 오피스의 컴퓨터라던지?
9,P,"2오피스에 컴퓨터 한대 그리고 회의가능하게 화이트보드가 있고, 업무환경의 소프트웨어..."


In [7]:
train_data

[['DSJY',
  '"자 저희는 유니스트 소속 대학원 수업 CONTEXTUAL DESGIN 을 수강하고 있는 이지영입니다. 저희는 학생창업자로서 겪는 여러가지 경험을 자세히 듣고자하며, 특히 업무 진행, 갈등 중재, 활용하는 테크놀로지에 대해 관심이 많아서 이렇게 인터뷰를 진행하게 되었습니다. 인터뷰는 약 30분정도 진행할 것 같고요. 그럼 시작 할꼐요."'],
 ['P2', '네'],
 ['DSJY', '보나는 어떤 집단인지 궁금합니다'],
 ['P2',
  '"우리는 습관에 대해서 항상하고 사람들이 움직이는 방법에 대해서 건강한 습관을 가지고 내가 원하는것, 보나라는 이름이 원래 보고싶은 나라는 뜻이에요. 내가 원하는것과 하고싶어하는 것을 습관을 통해서 이루어간다는 비전을 가지고 있고요. 지금 하는 것은 여러가지 시도를 해보다가 현재는 건강음식 건강음료 이런것을 유니스트의 학생들이 건강하게 살면서 할 수 있도록 지금 준비중에 있습니다."'],
 ['DSJY', '굉장히 좋은 취지인것 같습니다'],
 ['P2', '그렇죠? 우리팀에서 일할래?'],
 ['DSJY', '"돈주면..., 보나의 규모가 어느정도인가요?"'],
 ['P2',
  '"규모라는게 너무 작아서 규모라고 할 정도가 아니고 정말 작아서 그냥 1인기업, 스타트업이라고 생각하시면 될것 같습니다. 풀타임 워커가 3명, 하프타임이 1명, 풀타임이라는것은 9시부터 나와서 9시부터 5시반까지 지키는 사람들 보통은 그 시간 이상까지 있고요. 나머지는 수업이 있고 갔다오고 그러고요. 매출이나 그런게 없다보니 규모로 인원 규모 밖에 말씀을 못드립니다."'],
 ['DSJY', '업무환경에 대해서 말씀해 주실수 있나요? 오피스의 컴퓨터라던지?'],
 ['P',
  '"2오피스에 컴퓨터 한대 그리고 회의가능하게 화이트보드가 있고, 업무환경의 소프트웨어같은 쪽에서는 잔디,슬랙 이런것 중에서 저희는 잔디를 주로 사용을 하고요. 카톡과는 다른 업무용 메신저를 써서 이제 엄무시간 동안에는 될수 있도록 카톡을 안들어

In [8]:
print(len(train_data))
print(len(train_data[0]))

1732
2


In [9]:
from konlpy.tag import Twitter 
pos_tagger = Twitter()
def tokenize(doc):
    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]

In [10]:
train_docs = [(tokenize(row[1]), row[0]) for row in train_data] 

from pprint import pprint 
pprint(train_docs[5])

(['그렇다/Adjective',
  '?/Punctuation',
  '우리/Noun',
  '팀/Noun',
  '에서/Josa',
  '일/Noun',
  '하다/Verb',
  '?/Punctuation'],
 'P2')


In [11]:
pprint(train_docs)

[(['"/Punctuation',
   '자/Noun',
   '저희/Noun',
   '는/Josa',
   '유니스트/Noun',
   '소속/Noun',
   '대학원/Noun',
   '수업/Noun',
   'CONTEXTUAL/Alpha',
   'DESGIN/Alpha',
   '을/Josa',
   '수강/Noun',
   '하고/Josa',
   '있다/Adjective',
   '이지영/Noun',
   '이다/Adjective',
   './Punctuation',
   '저희/Noun',
   '는/Josa',
   '학생/Noun',
   '창업/Noun',
   '자/Suffix',
   '로서/Noun',
   '겪다/Verb',
   '여러가지/Noun',
   '경험/Noun',
   '을/Josa',
   '자세하다/Adjective',
   '듣다/Verb',
   '자하/Noun',
   '며/Josa',
   ',/Punctuation',
   '특히/Adverb',
   '업무/Noun',
   '진행/Noun',
   ',/Punctuation',
   '갈등/Noun',
   '중재/Noun',
   ',/Punctuation',
   '활용/Noun',
   '하다/Verb',
   '테크놀로지/Noun',
   '에/Josa',
   '대해/Noun',
   '관심/Noun',
   '이/Josa',
   '많다/Adjective',
   '이렇게/Adverb',
   '인터뷰/Noun',
   '를/Josa',
   '진행/Noun',
   '하다/Verb',
   '되어다/Verb',
   './Punctuation',
   '인터뷰/Noun',
   '는/Josa',
   '약/Noun',
   '30/Number',
   '분/Noun',
   '정도/Noun',
   '진행/Noun',
   '하다/Verb',
   '것/Noun',
   '같다/Adjective',
   './Punctuation',


   '어렵다/Adjective',
   ',/Punctuation',
   '그래서/Adverb',
   '지금/Noun',
   '당장/Noun',
   '우리/Noun',
   '옆/Noun',
   '에/Josa',
   '있다/Adjective',
   '사람/Noun',
   '들/Suffix',
   '을/Josa',
   '위/Noun',
   '하다/Verb',
   '서비스/Noun',
   '를/Josa',
   '만들다/Verb',
   '내자/Noun',
   '라고/Josa',
   '하다/Verb',
   '목표/Noun',
   '이고/Josa',
   '그리고/Conjunction',
   '이렇게/Adverb',
   '진행/Noun',
   '이/Josa',
   '되다/Verb',
   '건강하다/Adjective',
   '음식/Noun',
   '을/Josa',
   '아침/Noun',
   '에/Josa',
   '만들다/Verb',
   '하다/Verb',
   '있다/Adjective',
   '."/Punctuation'],
  'P2 '),
 (['멋있다/Adjective',
   './Punctuation',
   '그러면/Adverb',
   '아깝다/Adjective',
   '그/Noun',
   '플랭크/Noun',
   '를/Josa',
   '하다/Verb',
   '사람/Noun',
   '들/Suffix',
   '을/Josa',
   '교내/Noun',
   '에서/Josa',
   '모으다/Verb',
   '수/Noun',
   '있다/Adjective',
   '보나/Noun',
   '의/Josa',
   '장점/Noun',
   '이겠다/Verb'],
  'DSJY '),
 (['"/Punctuation',
   '우리/Noun',
   '는/Josa',
   '원래/Noun',
   '는/Josa',
   '플랭크/Noun',
   '를/Josa',
   '하다/Verb',
   '하

   '?/Punctuation',
   '다/Adverb',
   '나오다/Verb',
   '하다/Verb',
   '좋다/Adjective',
   ',/Punctuation',
   '그렇다/Adjective',
   '좋다/Adjective',
   ',/Punctuation',
   '그게/Noun',
   '자다/Verb',
   '안/Noun',
   '되다/Verb',
   '경우/Noun',
   '"/Punctuation'],
  'P2 '),
 (['아깝다/Adjective',
   '협업툴/Noun',
   '로/Josa',
   '잔디/Noun',
   '와/Josa',
   '카톡/Noun',
   '이야기/Noun',
   '를/Josa',
   '해주다/Verb',
   '그것/Noun',
   '외/Noun',
   '에/Josa',
   '따로/Adverb',
   '쓰다/Verb',
   '온라인/Noun',
   '툴/Noun',
   '이/Josa',
   '../Punctuation'],
  'DSJY'),
 (['그건/Adverb', '페이스북/Noun'], 'P2 '),
 (['페이스북/Noun', '이/Suffix', '요/Josa', '?/Punctuation'], 'DSJY '),
 (['우리/Noun',
   '가/Josa',
   '쓰다/Verb',
   '페이스북/Noun',
   '그룹/Noun',
   '이/Josa',
   '있다/Adjective',
   '그건/Adverb',
   '거기/Noun',
   '에/Josa',
   '좋다/Adjective',
   '글/Noun',
   '같다/Adjective',
   '공유/Noun',
   '를/Josa',
   '하다/Verb',
   '다/Adverb',
   '보다/Verb',
   '있다/Adjective',
   '하다/Verb',
   './Punctuation',
   '내/Noun',
   '가/Josa',
   '신선하다/Adj

   '그/Determiner',
   '쪽/Noun',
   '은/Josa',
   '이제/Noun',
   '컨텍/Noun',
   '에/Josa',
   '대한/Noun',
   '부분/Noun',
   '이/Josa',
   '좀/Noun',
   '어렵다/Adjective',
   '그래서/Adverb',
   '이제/Noun',
   '사실/Noun',
   '저희/Noun',
   '도/Josa',
   '이제/Noun',
   '조금/Noun',
   '회사/Noun',
   '../Punctuation',
   '단순하다/Adjective',
   '이제/Noun',
   '그냥/Noun',
   '지금/Noun',
   '은/Josa',
   '팀/Noun',
   '느낌/Noun',
   '인데/Josa',
   ',/Punctuation',
   '진짜/Noun',
   '이제/Noun',
   '회사/Noun',
   '로/Josa',
   '크다/Verb',
   '투자/Noun',
   '같다/Adjective',
   '이제/Noun',
   '받다/Verb',
   ',/Punctuation',
   '받다/Verb',
   '이제/Noun',
   ',/Punctuation',
   '또/Noun',
   '어느/Adverb',
   '정도/Noun',
   '성과/Noun',
   '가/Josa',
   ',/Punctuation',
   '확/Noun',
   '눈/Noun',
   '에/Josa',
   '띄다/Verb',
   '성과/Noun',
   '가/Josa',
   '있다/Adjective',
   '되다/Verb',
   ',/Punctuation',
   '사실/Noun',
   '은/Josa',
   '또/Noun',
   '그게/Noun',
   '또/Noun',
   '진행/Noun',
   '이/Josa',
   '진행/Noun',
   '이/Josa',
   '되다/Verb',
   '또/Noun',

   '것/Noun',
   '이다/Josa',
   '라는/Josa',
   '생각/Noun',
   '하다/Verb',
   '있다/Adjective',
   '조금/Noun',
   '이/Noun',
   '게/Josa',
   '시간/Noun',
   '이/Josa',
   '지나다/Verb',
   '지나다/Verb',
   ',/Punctuation',
   '지나다/Verb',
   '이/Noun',
   '게/Josa',
   '조금/Noun',
   '옅어/Noun',
   '지는/Josa',
   '느낌/Noun',
   '이/Josa',
   '있다/Adjective',
   '같다/Adjective',
   './Punctuation',
   '예/Noun',
   '…/Foreign',
   '"/Punctuation'],
  'I1'),
 (['네/Noun',
   './Punctuation',
   '그/Noun',
   '다음/Noun',
   '은/Josa',
   '저희/Noun',
   '가/Josa',
   '창업/Noun',
   '팀/Noun',
   '내부/Noun',
   '에서/Josa',
   '의사소통/Noun',
   '을/Josa',
   '하다/Verb',
   '그/Noun',
   '실태/Noun',
   '?/Punctuation',
   '에/Josa',
   '대하다/Verb',
   '조금/Noun',
   '조사/Noun',
   '를/Josa',
   '해보다/Verb',
   '싶다/Verb',
   '혹시/Noun',
   '현/Noun',
   '시점/Noun',
   '?/Punctuation',
   '그/Noun',
   '../Punctuation',
   '오픈/Noun',
   '아레나/Noun',
   '에게/Josa',
   '가장/Noun',
   '중요하다/Adjective',
   '마/Noun',
   '일드/Noun',
   '스톤/Noun',
   '이나/Josa

...

   '그렇다/Adjective',
   '일정/Noun',
   '관리/Noun',
   '를/Josa',
   '어떻다/Adjective',
   '진행/Noun',
   '하다/Verb',
   ',/Punctuation',
   '온라인/Noun',
   '으로/Josa',
   '하다/Verb',
   '오프라인/Noun',
   '으로/Josa',
   '하다/Verb',
   '."/Punctuation'],
  'DSJY:  '),
 (['"/Punctuation',
   '스케쥴/Noun',
   '관리/Noun',
   '는/Josa',
   '같이/Josa',
   '쓰다/Verb',
   '구글/Noun',
   ',/Punctuation',
   '각자/Noun',
   '스케쥴/Noun',
   '을/Josa',
   '적다/Verb',
   '구글/Noun',
   '캘린더/Noun',
   '가/Josa',
   '있다/Adjective',
   '해/Noun',
   '요/Josa',
   './Punctuation',
   '음/Noun',
   '근데/Adverb',
   '크게/Noun',
   '활용/Noun',
   '되다/Verb',
   '본적/Noun',
   '은/Josa',
   '없다/Adjective',
   '것/Noun',
   '같다/Adjective',
   './Punctuation',
   '일단/Noun',
   '월요일/Noun',
   '밤/Noun',
   '으로/Josa',
   '잡히다/Verb',
   '기다/Verb',
   '정기/Noun',
   '모임/Noun',
   '은/Josa',
   '잡히다/Verb',
   '일일/Noun',
   '보고/Noun',
   '는/Josa',
   '자기/Noun',
   '가/Josa',
   '쓰다/Verb',
   '올리다/Verb',
   '것/Noun',
   '이기/Noun',
   '때문/Noun',
   '에/Josa',


   '그러면/Adverb',
   '커뮤니케이션/Noun',
   '이/Josa',
   '온라인/Noun',
   ',/Punctuation',
   '오프라인/Noun',
   '두/Determiner',
   '가지/Noun',
   '가/Josa',
   '있다/Adjective',
   '?/Punctuation',
   '온라인/Noun',
   '커뮤니케이션/Noun',
   '이/Josa',
   '더/Noun',
   '편하다/Adjective',
   '경우/Noun',
   '는/Josa',
   '언제/Noun',
   '가/Josa',
   '있다/Adjective',
   '?"/Punctuation'],
  'DSJY: '),
 (['"/Punctuation',
   '음/Noun',
   '온라인/Noun',
   '이/Josa',
   '편하다/Adjective',
   '경우/Noun',
   '..../Punctuation',
   '저/Noun',
   '는/Josa',
   '항상/Noun',
   '오프라인/Noun',
   '이/Josa',
   '더/Noun',
   '편하다/Adjective',
   ',/Punctuation',
   '온라인/Noun',
   '이/Josa',
   '더/Noun',
   '편하다/Adjective',
   '경우/Noun',
   '는/Josa',
   '제/Noun',
   '가/Josa',
   '기숙사/Noun',
   '에서/Josa',
   '쉬다/Verb',
   '데이터/Noun',
   '를/Josa',
   '전달/Noun',
   '하다/Verb',
   '하다/Verb',
   '떄/Noun',
   '?/Punctuation',
   '이다/Verb',
   '같다/Adjective',
   './Punctuation',
   '뭐/Noun',
   '지/Josa',
   '온라인/Noun',
   '의/Josa',
   '장점/Noun',
   '인걸/J

   '."/Punctuation'],
  'JH04. '),
 (['학생/Noun',
   '창업/Noun',
   '자/Suffix',
   '로서/Noun',
   '의/Josa',
   '장점/Noun',
   '은/Josa',
   '?/Punctuation'],
  'Q. '),
 (['"/Punctuation',
   '한국/Noun',
   '에서/Josa',
   '학생/Noun',
   '창업/Noun',
   '자/Suffix',
   '로서/Noun',
   '의/Josa',
   '장점/Noun',
   '은/Josa',
   '분명하다/Adjective',
   '있다/Adjective',
   './Punctuation',
   '예/Noun',
   '를/Josa',
   '들다/Verb',
   '특허/Noun',
   '라/Josa',
   '던지다/Verb',
   '디자인권/Noun',
   '같다/Adjective',
   '신청/Noun',
   '하다/Verb',
   '때/Noun',
   '도/Josa',
   '학생/Noun',
   '이면/Noun',
   '은/Josa',
   '우대/Noun',
   '를/Josa',
   '많이/Adverb',
   '받다/Verb',
   './Punctuation',
   '그리고/Conjunction',
   '학생/Noun',
   '때/Noun',
   '예/Noun',
   '를/Josa',
   '들다/Verb',
   '가시/Noun',
   '적/Suffix',
   '인/Josa',
   '성과/Noun',
   '를/Josa',
   '내다/Verb',
   '면/Noun',
   '학교/Noun',
   '에서/Josa',
   '지원/Noun',
   '을/Josa',
   '받다/Verb',
   '수가/Noun',
   '있다/Adjective',
   './Punctuation',
   '그러니까/Adverb',
   '왜냐면/Noun',
   

   '맞추다/Verb',
   './Punctuation',
   '저/Noun',
   '랑/Josa',
   '하다/Verb',
   '업무/Noun',
   '에/Josa',
   '덜/Noun',
   '신경/Noun',
   '이/Josa',
   '쓰다/Verb',
   '그렇다/Adjective',
   '부분/Noun',
   '들/Suffix',
   '?/Punctuation',
   '저/Noun',
   '는/Josa',
   '이제/Noun',
   '주/Noun',
   '니까/Josa',
   './Punctuation',
   '그/Noun',
   '균형/Noun',
   '을/Josa',
   '맞추다/Verb',
   '중요하다/Adjective',
   '것/Noun',
   '같다/Adjective',
   './Punctuation',
   '"/Punctuation'],
  'JH04.  '),
 (['우선/Noun',
   '순위/Noun',
   '처럼/Josa',
   '자기/Noun',
   '는/Josa',
   '이/Noun',
   '게/Josa',
   '중요하다/Adjective',
   '생각/Noun',
   '하다/Verb',
   '사람/Noun',
   '들/Suffix',
   '에게/Josa',
   '받아들이다/Verb',
   '않다/Verb',
   '때/Noun',
   '가/Josa',
   '있다/Adjective',
   './Punctuation',
   '그렇다/Adjective',
   '경우/Noun',
   '에는/Josa',
   '어떻다/Adjective',
   '하다/Verb',
   '?/Punctuation'],
  'Q. '),
 (['그냥/Noun',
   '그냥/Noun',
   '대화/Noun',
   '를/Josa',
   '하다/Verb',
   '수/Noun',
   '밖에/Josa',
   '없다/Adjective',
   './Punctuat

   '공감/Noun',
   '을/Josa',
   '하다/Verb',
   './Punctuation',
   '그리고/Conjunction',
   '그/Noun',
   '친구/Noun',
   '는/Josa',
   '원래/Noun',
   '자기/Noun',
   '가/Josa',
   '창업/Noun',
   '을/Josa',
   '하다/Verb',
   '하다/Verb',
   '친구/Noun',
   '에요/Josa',
   './Punctuation',
   '창업/Noun',
   '을/Josa',
   '하다/Verb',
   '친구/Noun',
   '를/Josa',
   '제/Noun',
   '가/Josa',
   '땡기다/Verb',
   '오다/Verb',
   '그/Noun',
   '친구/Noun',
   '는/Josa',
   '원래/Noun',
   '목표/Noun',
   '는/Josa',
   '자기/Noun',
   '창업/Noun',
   '이다/Verb',
   '때문/Noun',
   '에/Josa',
   '이쯤/Noun',
   '에서/Josa',
   '자기/Noun',
   '창업/Noun',
   '을/Josa',
   '하다/Verb',
   '저/Noun',
   '는/Josa',
   '더/Noun',
   '나은/Noun',
   '길/Noun',
   '을/Josa',
   '찾아가다/Verb',
   '맞다/Verb',
   '생각/Noun',
   '을/Josa',
   '한/Determiner',
   '거/Noun',
   '죠/Josa',
   './Punctuation',
   '그래서/Adverb',
   '자연/Noun',
   '스럽게/Josa',
   '이렇게/Adverb',
   '된거/Noun',
   '고/Josa',
   './Punctuation',
   '지금/Noun',
   '도/Josa',
   '굉장하다/Adjective',
   '자다/Verb',
   '

In [12]:
type(train_docs)

list

In [13]:
morph_cate = []
for item in train_docs:
    for i in item[0]:
        b = i.split('/')
        c = b[1]
        morph_cate.append(c)

In [14]:
remove_item = ('','Conjunction', 'Determiner', 'Exclamation','Josa','Punctuation','Suffix')

In [15]:
trained_docs_list = []

for i,item in enumerate(train_docs):
    dummy = []
    for indi in item[0]:
        if (indi.split('/')[1] not in remove_item):
            dummy.append(indi)
    trained_docs_list.append((dummy,item[1]))

In [16]:
trained_docs_list[:7]

[(['자/Noun',
   '저희/Noun',
   '유니스트/Noun',
   '소속/Noun',
   '대학원/Noun',
   '수업/Noun',
   'CONTEXTUAL/Alpha',
   'DESGIN/Alpha',
   '수강/Noun',
   '있다/Adjective',
   '이지영/Noun',
   '이다/Adjective',
   '저희/Noun',
   '학생/Noun',
   '창업/Noun',
   '로서/Noun',
   '겪다/Verb',
   '여러가지/Noun',
   '경험/Noun',
   '자세하다/Adjective',
   '듣다/Verb',
   '자하/Noun',
   '특히/Adverb',
   '업무/Noun',
   '진행/Noun',
   '갈등/Noun',
   '중재/Noun',
   '활용/Noun',
   '하다/Verb',
   '테크놀로지/Noun',
   '대해/Noun',
   '관심/Noun',
   '많다/Adjective',
   '이렇게/Adverb',
   '인터뷰/Noun',
   '진행/Noun',
   '하다/Verb',
   '되어다/Verb',
   '인터뷰/Noun',
   '약/Noun',
   '30/Number',
   '분/Noun',
   '정도/Noun',
   '진행/Noun',
   '하다/Verb',
   '것/Noun',
   '같다/Adjective',
   '그렇다/Adjective',
   '시작/Noun',
   '하다/Verb'],
  'DSJY'),
 (['네/Noun'], 'P2'),
 (['보나/Noun', '어떻다/Adjective', '집단/Noun', '궁금하다/Adjective'], 'DSJY'),
 (['우리/Noun',
   '습관/Noun',
   '대하다/Verb',
   '항상/Noun',
   '사람/Noun',
   '움직이다/Verb',
   '방법/Noun',
   '대하다/Verb',
   '건강하다/Adjective'

In [17]:
trained_docs_list

[(['자/Noun',
   '저희/Noun',
   '유니스트/Noun',
   '소속/Noun',
   '대학원/Noun',
   '수업/Noun',
   'CONTEXTUAL/Alpha',
   'DESGIN/Alpha',
   '수강/Noun',
   '있다/Adjective',
   '이지영/Noun',
   '이다/Adjective',
   '저희/Noun',
   '학생/Noun',
   '창업/Noun',
   '로서/Noun',
   '겪다/Verb',
   '여러가지/Noun',
   '경험/Noun',
   '자세하다/Adjective',
   '듣다/Verb',
   '자하/Noun',
   '특히/Adverb',
   '업무/Noun',
   '진행/Noun',
   '갈등/Noun',
   '중재/Noun',
   '활용/Noun',
   '하다/Verb',
   '테크놀로지/Noun',
   '대해/Noun',
   '관심/Noun',
   '많다/Adjective',
   '이렇게/Adverb',
   '인터뷰/Noun',
   '진행/Noun',
   '하다/Verb',
   '되어다/Verb',
   '인터뷰/Noun',
   '약/Noun',
   '30/Number',
   '분/Noun',
   '정도/Noun',
   '진행/Noun',
   '하다/Verb',
   '것/Noun',
   '같다/Adjective',
   '그렇다/Adjective',
   '시작/Noun',
   '하다/Verb'],
  'DSJY'),
 (['네/Noun'], 'P2'),
 (['보나/Noun', '어떻다/Adjective', '집단/Noun', '궁금하다/Adjective'], 'DSJY'),
 (['우리/Noun',
   '습관/Noun',
   '대하다/Verb',
   '항상/Noun',
   '사람/Noun',
   '움직이다/Verb',
   '방법/Noun',
   '대하다/Verb',
   '건강하다/Adjective'

In [18]:
tokens = [t for d in trained_docs_list for t in d[0]] 
print(len(tokens))

47931


In [19]:
import nltk
text = nltk.Text(tokens, name='NMSC') 
print(text)

<Text: NMSC>


In [20]:
print(len(text.tokens))
print(len(set(text.tokens)))
pprint(text.vocab().most_common(10))

47931
3945
[('하다/Verb', 4163),
 ('있다/Adjective', 1863),
 ('그렇다/Adjective', 841),
 ('되다/Verb', 740),
 ('같다/Adjective', 646),
 ('것/Noun', 645),
 ('그/Noun', 421),
 ('어떻다/Adjective', 391),
 ('때/Noun', 387),
 ('이렇다/Adjective', 347)]


In [21]:
text.collocations()

수/Noun 있다/Adjective; 것/Noun 같다/Adjective; 안/Noun 되다/Verb; 생각/Noun
하다/Verb; 예/Noun 들다/Verb; 이야기/Noun 하다/Verb; 사용/Noun 하다/Verb;
같다/Adjective 경우/Noun; 오픈/Noun 아레나/Noun; 어느/Adverb 정도/Noun; 예/Noun
들어서다/Verb; 위/Noun 하다/Verb; 온라인/Noun 커뮤니케이션/Noun; 하다/Verb 때/Noun;
학생/Noun 창업/Noun; 일/Noun 하다/Verb; 수도/Noun 있다/Adjective; 하다/Verb 수/Noun;
시작/Noun 하다/Verb; 고민/Noun 거리/Noun


In [22]:
trained_docs_list[5]

(['그렇다/Adjective', '우리/Noun', '팀/Noun', '일/Noun', '하다/Verb'], 'P2')

In [23]:
from collections import namedtuple
TaggedDocument = namedtuple('TaggedDocument', 'words tags')
tagged_train_docs = [TaggedDocument(d, [c]) for d, c in trained_docs_list]

In [24]:
from gensim.models import doc2vec

In [25]:
model = gensim.models.doc2vec.Doc2Vec(size=20, min_count=3, iter=20)
model.build_vocab(tagged_train_docs)
model.train(tagged_train_docs, total_examples=model.corpus_count, epochs=model.iter)

722160

In [26]:
model.save_word2vec_format('doc_tensor.w2v', doctag_vec=True, word_vec=False)

In [27]:
with open('text_metadata.tsv','w') as w:
    w.write('TEXT\tID\n')
    for i,j in zip(df.TEXT, df.ID):
        w.write("%s\t%s\n" % (i,j))

In [28]:
import pandas as pd
import re
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation
from gensim.models import ldamodel
from gensim.corpora.dictionary import Dictionary

import multiprocessing
cores = multiprocessing.cpu_count()
cores

4

In [29]:
doc_vectorizer = doc2vec.Doc2Vec(
    dm=0,            # PV-DBOW / default 1
    dbow_words=1,    # w2v simultaneous with DBOW d2v / default 0
    window=8,        # distance between the predicted word and context words 
    size=300,        # vector size 
    alpha=0.025,     # learning-rate
    seed=1234,
    min_count=5,    # ignore with freq lower
    min_alpha=0.025, # min learning-rate
    workers=cores,   # multi cpu
    hs = 1,          # hierarchical softmax / default 0
    negative = 10,   # negative sampling / default 5
    )

doc_vectorizer.build_vocab(tagged_train_docs)
print(str(doc_vectorizer))

Doc2Vec(dbow+w,d300,n10,hs,w8,mc5,s0.001,t4)


In [30]:
doc_vectorizer.save('Doc2Vec(dbow+w,d300,n10,hs,w8,mc20,s0.001,t24).model')

In [31]:
doc_vectorizer = doc2vec.Doc2Vec.load('Doc2Vec(dbow+w,d300,n10,hs,w8,mc20,s0.001,t24).model')

In [32]:
train_x = [doc_vectorizer.infer_vector(doc.words) for doc in tagged_train_docs]
train_y = [doc.tags[0] for doc in tagged_train_docs]
len(train_x)

1732

In [33]:
import numpy as np

In [34]:
train_x_np = np.asarray(train_x)
train_y_np = np.asarray(train_y)

In [35]:
type(train_x_np[0,0])

numpy.float32

In [36]:
train_x_np.shape

(1732, 300)

In [37]:
train_y_np.shape

(1732,)

In [38]:
len(train_x_np)

1732

In [39]:
train_x_np

array([[-0.001,  0.   , -0.001, ...,  0.   ,  0.002,  0.001],
       [ 0.   , -0.001,  0.001, ..., -0.001,  0.002,  0.001],
       [ 0.001,  0.001,  0.   , ...,  0.001,  0.   ,  0.001],
       ..., 
       [-0.001, -0.001,  0.   , ...,  0.001,  0.   , -0.001],
       [-0.001,  0.001,  0.001, ...,  0.001,  0.001,  0.001],
       [ 0.001,  0.001,  0.   , ..., -0.001,  0.001,  0.001]], dtype=float32)

In [40]:
type(train_x_np)

numpy.ndarray

In [41]:
trained_list = []

for i,item in enumerate(trained_docs_list):
    dummy = []
    for indi in item[0]:
        dummy.append(indi.split('/')[0])
    trained_list.append(dummy)
    
trained_list

[['자',
  '저희',
  '유니스트',
  '소속',
  '대학원',
  '수업',
  'CONTEXTUAL',
  'DESGIN',
  '수강',
  '있다',
  '이지영',
  '이다',
  '저희',
  '학생',
  '창업',
  '로서',
  '겪다',
  '여러가지',
  '경험',
  '자세하다',
  '듣다',
  '자하',
  '특히',
  '업무',
  '진행',
  '갈등',
  '중재',
  '활용',
  '하다',
  '테크놀로지',
  '대해',
  '관심',
  '많다',
  '이렇게',
  '인터뷰',
  '진행',
  '하다',
  '되어다',
  '인터뷰',
  '약',
  '30',
  '분',
  '정도',
  '진행',
  '하다',
  '것',
  '같다',
  '그렇다',
  '시작',
  '하다'],
 ['네'],
 ['보나', '어떻다', '집단', '궁금하다'],
 ['우리',
  '습관',
  '대하다',
  '항상',
  '사람',
  '움직이다',
  '방법',
  '대하다',
  '건강하다',
  '습관',
  '가지',
  '내',
  '원',
  '하다',
  '보나',
  '이름',
  '원래',
  '보다',
  '나라',
  '뜻',
  '내',
  '원',
  '하다',
  '하다',
  '것',
  '습관',
  '통',
  '하다',
  '이루다',
  '간다',
  '비다',
  '가지',
  '있다',
  '지금',
  '하다',
  '것',
  '여러가지',
  '시도',
  '해보다',
  '현재',
  '건강',
  '음식',
  '건강',
  '음료',
  '이렇다',
  '유니스트',
  '학생',
  '건강하다',
  '살다',
  '하다',
  '수',
  '있다',
  '지금',
  '준비',
  '있다'],
 ['굉장하다', '좋다', '취지', '것', '같다'],
 ['그렇다', '우리', '팀', '일', '하다'],
 ['돈', '주다', '보나', '규모',

In [42]:
dictionary = Dictionary(trained_list)
dictionary.filter_extremes(no_below=2, no_above=0.3)
corpus = [dictionary.doc2bow(text) for text in trained_list]

In [43]:
num_topics = 10
chunksize = 2000
passes = 50
iterations = 200
eval_every = None

# Train model
model = ldamodel.LdaModel(corpus=corpus, id2word=dictionary, chunksize=chunksize, alpha='auto', eta='auto', iterations=iterations, num_topics=num_topics, passes=passes, eval_every=eval_every)

In [44]:
all_topics = model.get_document_topics(corpus, minimum_probability=0)
all_topics[0]

[(0, 0.00095964176855854527),
 (1, 0.00068543402345730756),
 (2, 0.00060470324030049364),
 (3, 0.00063616018323206351),
 (4, 0.00066192941940241504),
 (5, 0.99292546505748569),
 (6, 0.00065819726612046118),
 (7, 0.00065423263044340194),
 (8, 0.0014930848865263215),
 (9, 0.0007211515244733074)]

In [45]:
with open('doc_lda_tensor.tsv','w') as w:
    for doc_topics in all_topics:
        for topics in doc_topics:
            w.write(str(topics[1])+ "\t")
        w.write("\n")
        
# create file for metadata
with open('doc_lda_metadata.tsv','w') as w:
    w.write('TEXT\ID\n')
    for j, k in zip(df.TEXT, df.ID):
        w.write("%s\t%s\n" % (j, k))

In [46]:
tensors = []
for doc_topics in all_topics:
    doc_tensor = []
    for topic in doc_topics:
        if round(topic[1], 3) > 0:
            doc_tensor.append((topic[0], float(round(topic[1], 3))))
    # sort topics according to highest probabilities
    doc_tensor = sorted(doc_tensor, key=lambda x: x[1], reverse=True)
    # store vectors to add in metadata file
    tensors.append(doc_tensor[:5])

# overwrite metadata file
i=0
with open('doc_lda_metadata.tsv','w') as w:
    w.write('Titles\tGenres\n')
    for j,k in zip(df.TEXT, df.ID):
        w.write("%s\t%s\n" % (''.join((str(j), str(tensors[i]))),k))
        i+=1

In [47]:
model.show_topic(topicid=0, topn=20)

[('네', 0.018200813733381026),
 ('없다', 0.013043087570829809),
 ('그', 0.012653711455527983),
 ('것', 0.012500355617777143),
 ('같다', 0.012460254259235552),
 ('되다', 0.012155027650346898),
 ('때', 0.01132727467971272),
 ('팀', 0.010787288662338501),
 ('지금', 0.009649854185410393),
 ('다', 0.0095414513446980872),
 ('명', 0.0083050497499655238),
 ('일', 0.0082734806994102569),
 ('저희', 0.0079440766002289772),
 ('수', 0.0075348624227620578),
 ('이제', 0.0074277452295890114),
 ('아니다', 0.0073185461004717832),
 ('안', 0.0072944420559028214),
 ('그래서', 0.0065073159760128734),
 ('어떻다', 0.0064848159245616942),
 ('회의', 0.0062866580434190944)]

In [48]:
import pyLDAvis.gensim

viz = pyLDAvis.gensim.prepare(model, corpus, dictionary)
pyLDAvis.display(viz)

/Users/mac/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
